##### Copyright 2020 The TensorFlow Hub Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/text/tutorials/classify_text_with_bert"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/classify_text_with_bert.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/docs/tutorials/classify_text_with_bert.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/docs/tutorials/classify_text_with_bert.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
  <td>
    <a href="https://tfhub.dev/google/collections/bert/1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png" />See TF Hub model</a>
  </td>
</table>

## Setup


In [ ]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U "tensorflow-text==2.11.*"
!pip install -q tf-models-official==2.11.0
!pip install tensorflow_hub

In [ ]:
import os
import shutil
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import validation_curve
from sklearn.metrics import make_scorer
from sklearn import metrics as mt
from sklearn.metrics import f1_score as f1
import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [ ]:
print(tf.config.list_physical_devices('GPU'))

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 16
seed = 42

### Restructure input

In [ ]:
def remove_non_ascii(sentence):
    return ''.join(char for char in sentence if ord(char) < 128)
    
def html_Filter(sentence):
    sentence = BeautifulSoup(sentence, "lxml").text
    #print("after html_Filter",sentence)
    
    return sentence

In [ ]:
def delete_files_in_directory(directory_path):
    try:
        files = os.listdir(directory_path)
        for file in files:
            file_path = os.path.join(directory_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("All files deleted successfully.")
    except OSError:
        print("Error occurred while deleting files.")

In [ ]:
def create_folder(class_list,file_path,df_data,col = "y_ETD"): 
    for name in class_list:
        newpath = file_path+name+'/' 
        if not os.path.exists(newpath):
            os.makedirs(newpath)
        else:
            delete_files_in_directory(newpath)
    
#     df_data[col] = list(1-df_data[col].values)
    
    for i, (sentence, label) in enumerate(zip(df_data["issue_clean"],df_data[col])):
        sentence = html_Filter(sentence)
        sentence = remove_non_ascii(sentence)
    
        #vlabel_list.append(label)
    
        with open(file_path+str(label)+'/'+str(i)+".txt","w",encoding="UTF-8") as f:
            f.write(sentence)

In [ ]:
def tfdata_unzip(
    tfdata: tf.data.Dataset,
    *,
    recursive: bool=False,
    eager_numpy: bool=False,
    num_parallel_calls: int=tf.data.AUTOTUNE,
):
    """
    Unzip a zipped tf.data pipeline.

    Args:
        tfdata: the :py:class:`tf.data.Dataset`
            to unzip.

        recursive: Set to ``True`` to recursively unzip
            multiple layers of zipped pipelines.
            Defaults to ``False``.

        eager_numpy: Set this to ``True`` to return
            Python lists of primitive types or
            :py:class:`numpy.array` objects. Defaults
            to ``False``.

        num_parallel_calls: The level of parallelism to
            each time we ``map()`` over a
            :py:class:`tf.data.Dataset`.

    Returns:
        Returns a Python list of either
             :py:class:`tf.data.Dataset` or NumPy
             arrays.
    """
    if isinstance(tfdata.element_spec, tf.TensorSpec):
        if eager_numpy:
            return list(tfdata.as_numpy_iterator())
        return tfdata
        
    
    def tfdata_map(i: int) -> list:
        return tfdata.map(
            lambda *cols: cols[i],
            deterministic=True,
            num_parallel_calls=num_parallel_calls,
        )

    if isinstance(tfdata.element_spec, tuple):
        num_columns = len(tfdata.element_spec)
        if recursive:
            return [
                tfdata_unzip(
                    tfdata_map(i),
                    recursive=recursive,
                    eager_numpy=eager_numpy,
                    num_parallel_calls=num_parallel_calls,
                )
                for i in range(num_columns)
            ]
        else:
            return [
                tfdata_map(i)
                for i in range(num_columns)
            ]

    raise ValueError(
        "Unknown tf.data.Dataset element_spec: " +
        str(tfdata.element_spec)
    )

## Dataset Generation for 10-fold

In [ ]:
train_ds_etd_list = []
val_ds_etd_list = []
test_ds_etd_list = []

train_ds_ps_list = []
val_ds_ps_list = []
test_ds_ps_list = []

In [ ]:
path = "../data/cross_validate_10/"
dir_list = sorted(os.listdir(path))

for folder in dir_list:
    train_etd_path = path+folder+"/train/train_bert_etd/"
    val_etd_path = path+folder+"/val/val_bert_etd/"
    test_etd_path = path+folder+"/test/test_bert_etd/"
    
    train_ps_path = path+folder+"/train/train_bert_ps/"
    val_ps_path = path+folder+"/val/val_bert_ps/"
    test_ps_path = path+folder+"/test/test_bert_ps/"
    
    df_train = pd.read_csv(path+folder+"/train.csv")
    df_val = pd.read_csv(path+folder+"/val.csv")
    df_test = pd.read_csv(path+folder+"/test.csv")
    
    df_etd_pattern_train = pd.read_csv(path+folder+"/train/etd_pattern_train.csv")
    df_ps_pattern_train = pd.read_csv(path+folder+"/train/ps_pattern_train.csv")
    df_etd_pattern_test = pd.read_csv(path+folder+"/test/etd_pattern_test.csv")
    df_ps_pattern_test = pd.read_csv(path+folder+"/test/ps_pattern_test.csv")
    df_etd_pattern_val = pd.read_csv(path+folder+"/val/etd_pattern_val.csv")
    df_ps_pattern_val = pd.read_csv(path+folder+"/val/ps_pattern_val.csv")
    
    class_list = sorted([str(folder) for folder in set(df_train["y_ETD"])])
    
#     create_folder(class_list,train_etd_path,df_train,col = "y_ETD")
#     create_folder(class_list,val_etd_path,df_val,col = "y_ETD")
#     create_folder(class_list,test_etd_path,df_test,col = "y_ETD")
    
#     create_folder(class_list,train_ps_path,df_train,col = "y_PS")
#     create_folder(class_list,val_ps_path,df_val,col = "y_PS")
#     create_folder(class_list,test_ps_path,df_test,col = "y_PS")
    
    X_etd_train = df_etd_pattern_train.values
    X_ps_train = df_ps_pattern_train.values
    X_etd_test = df_etd_pattern_test.values
    X_ps_test = df_ps_pattern_test.values
    X_etd_val = df_etd_pattern_val.values
    X_ps_val = df_ps_pattern_val.values
    
    #For ETD
    pattern_train_ds = tf.data.Dataset.from_tensor_slices(X_etd_train)
    pattern_val_ds = tf.data.Dataset.from_tensor_slices(X_etd_val)
    pattern_test_ds = tf.data.Dataset.from_tensor_slices(X_etd_test)
    
    train_ds_etd = tf.keras.utils.text_dataset_from_directory(
        train_etd_path,
        labels='inferred',
        label_mode = 'categorical',
        batch_size=batch_size,
        shuffle = False,
        seed=seed)

    # use tfdata_unzip() to separate input from labels
    input_train, input_labels = tfdata_unzip(train_ds_etd)
    model_inputs = tf.data.Dataset.zip(({"sentences":input_train.unbatch(), "patterns":pattern_train_ds}, input_labels.unbatch()))
    model_inputs = model_inputs.shuffle(len(df_train), reshuffle_each_iteration=False).batch(batch_size = batch_size).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    
    val_ds_etd = tf.keras.utils.text_dataset_from_directory(
        val_etd_path,
        labels='inferred',
        label_mode = 'categorical',
        batch_size=batch_size,
        shuffle = False,
        seed=seed)

    # use tfdata_unzip() to separate input from labels
    input_val, input_val_labels = tfdata_unzip(val_ds_etd)
    model_val = tf.data.Dataset.zip(({"sentences":input_val.unbatch(), "patterns":pattern_val_ds}, input_val_labels.unbatch()))
    model_val = model_val.shuffle(len(df_val), reshuffle_each_iteration=False).batch(batch_size = batch_size).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    
    test_ds_etd = tf.keras.utils.text_dataset_from_directory(
        test_etd_path,
        label_mode = 'categorical',
        shuffle = False,
        batch_size=batch_size)

    # use tfdata_unzip() to separate input from labels
    input_test, input_test_labels = tfdata_unzip(test_ds_etd)
    model_test = tf.data.Dataset.zip(({"sentences":input_test.unbatch(), "patterns":pattern_test_ds}, input_test_labels.unbatch()))
    model_test = model_test.batch(batch_size = batch_size).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

    train_ds_etd_list.append(model_inputs)
    val_ds_etd_list.append(model_val)
    test_ds_etd_list.append(model_test)
    
     #For PS
    pattern_train_ds = tf.data.Dataset.from_tensor_slices(X_ps_train)
    pattern_val_ds = tf.data.Dataset.from_tensor_slices(X_ps_val)
    pattern_test_ds = tf.data.Dataset.from_tensor_slices(X_ps_test)
    
    train_ds_ps = tf.keras.utils.text_dataset_from_directory(
        train_ps_path,
        labels='inferred',
        label_mode = 'categorical',
        batch_size=batch_size,
        shuffle = False,
        seed=seed)
    
    # use tfdata_unzip() to separate input from labels
    input_train, input_labels = tfdata_unzip(train_ds_ps)
    model_inputs = tf.data.Dataset.zip(({"sentences":input_train.unbatch(), "patterns":pattern_train_ds}, input_labels.unbatch()))
    model_inputs = model_inputs.shuffle(len(df_train), reshuffle_each_iteration=False).batch(batch_size = batch_size).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    
    val_ds_ps = tf.keras.utils.text_dataset_from_directory(
        val_ps_path,
        labels='inferred',
        label_mode = 'categorical',
        batch_size=batch_size,
        shuffle = False,
        seed=seed)
    
    # use tfdata_unzip() to separate input from labels
    input_val, input_val_labels = tfdata_unzip(val_ds_ps)
    model_val = tf.data.Dataset.zip(({"sentences":input_val.unbatch(), "patterns":pattern_val_ds}, input_val_labels.unbatch()))
    model_val = model_val.shuffle(len(df_val), reshuffle_each_iteration=False).batch(batch_size = batch_size).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

    test_ds_ps = tf.keras.utils.text_dataset_from_directory(
        test_ps_path,
        label_mode = 'categorical',
        shuffle = False,
        batch_size=batch_size)
    
    # use tfdata_unzip() to separate input from labels
    input_test, input_test_labels = tfdata_unzip(test_ds_ps)
    model_test = tf.data.Dataset.zip(({"sentences":input_test.unbatch(), "patterns":pattern_test_ds}, input_test_labels.unbatch()))
    model_test = model_test.batch(batch_size = batch_size).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

    train_ds_ps_list.append(model_inputs)
    val_ds_ps_list.append(model_val)
    test_ds_ps_list.append(model_test)

## Loading models from TensorFlow Hub

Here you can choose which BERT model you will load from TensorFlow Hub and fine-tune. There are multiple BERT models available.

  - [BERT-Base](https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3), [Uncased](https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3) and [seven more models](https://tfhub.dev/google/collections/bert/1) with trained weights released by the original BERT authors.
  - [Small BERTs](https://tfhub.dev/google/collections/bert/1) have the same general architecture but fewer and/or smaller Transformer blocks, which lets you explore tradeoffs between speed, size and quality.
  - [ALBERT](https://tfhub.dev/google/collections/albert/1): four different sizes of "A Lite BERT" that reduces model size (but not computation time) by sharing parameters between layers.
  - [BERT Experts](https://tfhub.dev/google/collections/experts/bert/1): eight models that all have the BERT-base architecture but offer a choice between different pre-training domains, to align more closely with the target task.
  - [Electra](https://tfhub.dev/google/collections/electra/1) has the same architecture as BERT (in three different sizes), but gets pre-trained as a discriminator in a set-up that resembles a Generative Adversarial Network (GAN).
  - BERT with Talking-Heads Attention and Gated GELU [[base](https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1), [large](https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_large/1)] has two improvements to the core of the Transformer architecture.

The model documentation on TensorFlow Hub has more details and references to the
research literature. Follow the links above, or click on the [`tfhub.dev`](http://tfhub.dev) URL
printed after the next cell execution.

The suggestion is to start with a Small BERT (with fewer parameters) since they are faster to fine-tune. If you like a small model but with higher accuracy, ALBERT might be your next option. If you want even better accuracy, choose
one of the classic BERT sizes or their recent refinements like Electra, Talking Heads, or a BERT Expert.

Aside from the models available below, there are [multiple versions](https://tfhub.dev/google/collections/transformer_encoders_text/1) of the models that are larger and can yield even better accuracy, but they are too big to be fine-tuned on a single GPU. You will be able to do that on the [Solve GLUE tasks using BERT on a TPU colab](https://www.tensorflow.org/text/tutorials/bert_glue).

You'll see in the code below that switching the tfhub.dev URL is enough to try any of these models, because all the differences between them are encapsulated in the SavedModels from TF Hub.

In [ ]:
#@title Choose a BERT model to fine-tune

bert_model_name = 'small_bert/bert_en_uncased_L-12_H-768_A-12'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'electra_large':
        'https://tfhub.dev/google/electra_large/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
    'roberta_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/jeongukjae/roberta_en_cased_L-12_H-768_A-12/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_large':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'roberta_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

## Define custom model

In [ ]:
def build_combined_model(feature_size, dropout = 0.2, num_class=2):
    input1 = tf.keras.layers.Input(shape=(), dtype=tf.string, name='sentences')
    input2 = tf.keras.layers.Input(shape=(feature_size,), dtype=tf.float32, name='patterns')
    
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(input1)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.concat([net, input2], 1)
    net = tf.keras.layers.Dropout(dropout)(net)
    
    net = tf.keras.layers.Dense(num_class, activation='softmax', name='classifier')(net)
    #return tf.keras.Model([input1,input2], net)
    return tf.keras.Model(inputs={'sentences': input1, 'patterns': input2}, outputs=net)
    #return tf.keras.Model(input1, net)

## Model training and evaluation

In [ ]:
import tensorflow_addons as tfa

loss = tf.keras.losses.CategoricalCrossentropy()
metrics = tfa.metrics.F1Score(num_classes =2, threshold=0.5)

In [ ]:
def create_optimizer(ds, epochs):
    steps_per_epoch = tf.data.experimental.cardinality(ds).numpy()
    num_train_steps = steps_per_epoch * epochs
    num_warmup_steps = int(0.1*num_train_steps)
    
    init_lr = 3e-5
    optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=0,
                                          optimizer_type='adamw')
    
    return optimizer

In [ ]:
def evaluation_report(classifier_model,dataset, files, df_data):
    y_prob = classifier_model.predict(dataset)
    predicted_category_index = [np.argmax(y_prob[i]) for i in range(len(y_prob))]
    y_pred = [class_list[index] for index in predicted_category_index]
    
    ## generate report
    label_pred = np.zeros(len(y_pred))

    for i, file in enumerate(files):
        sid = file[:-4]
        label_pred[int(sid)] = y_pred[i]
    
    return label_pred

In [ ]:
precision_list = []
recall_list = []
f1_list = []
test_data = []
ylabels = []
yhatlabels = []
dropout_list = np.linspace(0.2, 0.6, 5)
epochs = 10

for i,(train_ds,val_ds,test_ds) in enumerate(zip(train_ds_etd_list, val_ds_etd_list, test_ds_etd_list)):
    optimizer = create_optimizer(train_ds,epochs)
    max_clf = build_classifier_model(dropout = 0.2)
    max_clf.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)
    
    print(f'Training model with {tfhub_handle_encoder}')
    history = max_clf.fit(x=train_ds,
                        validation_data=val_ds,
                        verbose=0,
                        epochs=epochs)
    
    max_clf = None
    max_score = 0
    best_dropout = 0.2
    print(f'Training model with {tfhub_handle_encoder}')
    for rate in dropout_list:
        clf = build_classifier_model(dropout = rate)
        clf.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)
    
    
        history = clf.fit(x=train_ds,
                        validation_data=val_ds,
                        verbose=0,
                        epochs=epochs)
        
        curr_score = np.average(history.history["val_classifier1_f1_score"])
        if curr_score > max_score:
            max_score = curr_score
            max_clf = clf
            best_dropout = rate
    
    print("fold",(i+1),": best dropout rate is",best_dropout)
    test_etd_path = path+str(i)+"/test/test_bert_etd/"
    df_test = pd.read_csv(path+str(i)+"/test.csv")
    test_files = []
    for test_folder in class_list:
        file_path = test_etd_path+test_folder+"/"
        test_files.extend(sorted(os.listdir(file_path)))
    
    label_pred = evaluation_report(max_clf, test_ds,test_files,df_test)
    
    y_test = df_test["y_ETD"].values
    precison = mt.precision_score(y_test, label_pred)
    recall = mt.recall_score(y_test, label_pred)
    score = mt.f1_score(y_test, label_pred)
    
    test_data.extend(list(df_test["issue_clean"].values))
    ylabels.extend(list(y_test))
    yhatlabels.extend(list(label_pred))
    
    precision_list.append(precison)
    recall_list.append(recall)
    f1_list.append(score)

print("precision:",round(np.average(precision_list),3),"recall:",round(np.average(recall_list),3),"F1:",round(np.average(f1_list),3))

In [ ]:
df_final = pd.DataFrame()
df_final["issue"] = test_data
df_final["y"] = ylabels
df_final["y_pred"] = yhatlabels
df_final
df_final.to_csv("../data/BERT_ETD_pattern_result.csv",index =None)

In [ ]:
precision_list = []
recall_list = []
f1_list = []
test_data = []
ylabels = []
yhatlabels = []
dropout_list = np.linspace(0.2, 0.6, 5)
epochs = 10
    
for i,(train_ds,val_ds,test_ds) in enumerate(zip(train_ds_ps_list, val_ds_ps_list, test_ds_ps_list)):
    optimizer = create_optimizer(train_ds,epochs) 
    max_clf = build_classifier_model(dropout = 0.2)
    max_clf.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)
    
    print(f'Training model with {tfhub_handle_encoder}')
    history = max_clf.fit(x=train_ds,
                        validation_data=val_ds,
                        verbose=0,
                        epochs=epochs)
    max_clf = None
    max_score = 0
    best_dropout = 0.2
    print(f'Training model with {tfhub_handle_encoder}')
    for rate in dropout_list:
        clf = build_classifier_model(dropout = rate)
        clf.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)
    
    
        history = clf.fit(x=train_ds,
                        validation_data=val_ds,
                        verbose=0,
                        epochs=epochs)
        
        curr_score = np.average(history.history["val_classifier1_f1_score"])
        if curr_score > max_score:
            max_score = curr_score
            max_clf = clf
            best_dropout = rate
    
    print("fold",(i+1),": best dropout rate is",best_dropout)
    test_etd_path = path+str(i)+"/test/test_bert_ps/"
    df_test = pd.read_csv(path+str(i)+"/test.csv")
    test_files = []
    for test_folder in class_list:
        file_path = test_etd_path+test_folder+"/"
        test_files.extend(sorted(os.listdir(file_path)))
    
    label_pred = evaluation_report(max_clf, test_ds,test_files,df_test)
    
    y_test = df_test["y_PS"].values
    precison = mt.precision_score(y_test, label_pred)
    recall = mt.recall_score(y_test, label_pred)
    score = mt.f1_score(y_test, label_pred)
    
    test_data.extend(list(df_test["issue_clean"].values))
    ylabels.extend(list(y_test))
    yhatlabels.extend(list(label_pred))
    
    precision_list.append(precison)
    recall_list.append(recall)
    f1_list.append(score)

print("precision:",round(np.average(precision_list),3),"recall:",round(np.average(recall_list),3),"F1:",round(np.average(f1_list),3))

In [ ]:
df_final = pd.DataFrame()
df_final["issue"] = test_data
df_final["y"] = ylabels
df_final["y_pred"] = yhatlabels
df_final
df_final.to_csv("../data/pretrained/BERT_PS_pattern_result.csv",index =None)

In [ ]:
bert_model_name = 'roberta_en_cased_L-12_H-768_A-12'
tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

In [ ]:
precision_list = []
recall_list = []
f1_list = []
test_data = []
ylabels = []
yhatlabels = []
dropout_list = np.linspace(0.2, 0.6, 5)
epochs = 10

for i,(train_ds,val_ds,test_ds) in enumerate(zip(train_ds_etd_list, val_ds_etd_list, test_ds_etd_list)):
    optimizer = create_optimizer(train_ds,epochs)
    max_clf = build_classifier_model(dropout = 0.2)
    max_clf.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)
    
    print(f'Training model with {tfhub_handle_encoder}')
    history = max_clf.fit(x=train_ds,
                        validation_data=val_ds,
                        verbose=0,
                        epochs=epochs)
    
    max_clf = None
    max_score = 0
    best_dropout = 0.2
    print(f'Training model with {tfhub_handle_encoder}')
    for rate in dropout_list:
        clf = build_classifier_model(dropout = rate)
        clf.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)
    
    
        history = clf.fit(x=train_ds,
                        validation_data=val_ds,
                        verbose=0,
                        epochs=epochs)
        
        curr_score = np.average(history.history["val_classifier1_f1_score"])
        if curr_score > max_score:
            max_score = curr_score
            max_clf = clf
            best_dropout = rate
    
    print("fold",(i+1),": best dropout rate is",best_dropout)
    test_etd_path = path+str(i)+"/test/test_bert_etd/"
    df_test = pd.read_csv(path+str(i)+"/test.csv")
    test_files = []
    for test_folder in class_list:
        file_path = test_etd_path+test_folder+"/"
        test_files.extend(sorted(os.listdir(file_path)))
    
    label_pred = evaluation_report(max_clf, test_ds,test_files,df_test)
    
    y_test = df_test["y_ETD"].values
    precison = mt.precision_score(y_test, label_pred)
    recall = mt.recall_score(y_test, label_pred)
    score = mt.f1_score(y_test, label_pred)
    
    test_data.extend(list(df_test["issue_clean"].values))
    ylabels.extend(list(y_test))
    yhatlabels.extend(list(label_pred))
    
    precision_list.append(precison)
    recall_list.append(recall)
    f1_list.append(score)

print("precision:",round(np.average(precision_list),3),"recall:",round(np.average(recall_list),3),"F1:",round(np.average(f1_list),3))

In [ ]:
df_final = pd.DataFrame()
df_final["issue"] = test_data
df_final["y"] = ylabels
df_final["y_pred"] = yhatlabels
df_final
df_final.to_csv("../data/pretrained/Roberta_ETD_pattern_result.csv",index =None)

In [ ]:
precision_list = []
recall_list = []
f1_list = []
test_data = []
ylabels = []
yhatlabels = []
dropout_list = np.linspace(0.2, 0.6, 5)
epochs = 10
    
for i,(train_ds,val_ds,test_ds) in enumerate(zip(train_ds_ps_list, val_ds_ps_list, test_ds_ps_list)):
    optimizer = create_optimizer(train_ds,epochs)
    max_clf = build_classifier_model(dropout = 0.2)
    max_clf.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)
    
    print(f'Training model with {tfhub_handle_encoder}')
    history = max_clf.fit(x=train_ds,
                        validation_data=val_ds,
                        verbose=0,
                        epochs=epochs)
    max_clf = None
    max_score = 0
    best_dropout = 0.2
    print(f'Training model with {tfhub_handle_encoder}')
    for rate in dropout_list:
        clf = build_classifier_model(dropout = rate)
        clf.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)
    
    
        history = clf.fit(x=train_ds,
                        validation_data=val_ds,
                        verbose=0,
                        epochs=epochs)
        
        curr_score = np.average(history.history["val_classifier1_f1_score"])
        if curr_score > max_score:
            max_score = curr_score
            max_clf = clf
            best_dropout = rate
    
    print("fold",(i+1),": best dropout rate is",best_dropout)
    test_etd_path = path+str(i)+"/test/test_bert_ps/"
    df_test = pd.read_csv(path+str(i)+"/test.csv")
    test_files = []
    for test_folder in class_list:
        file_path = test_etd_path+test_folder+"/"
        test_files.extend(sorted(os.listdir(file_path)))
    
    label_pred = evaluation_report(max_clf, test_ds,test_files,df_test)
    
    y_test = df_test["y_PS"].values
    precison = mt.precision_score(y_test, label_pred)
    recall = mt.recall_score(y_test, label_pred)
    score = mt.f1_score(y_test, label_pred)
    
    test_data.extend(list(df_test["issue_clean"].values))
    ylabels.extend(list(y_test))
    yhatlabels.extend(list(label_pred))
    
    precision_list.append(precison)
    recall_list.append(recall)
    f1_list.append(score)

print("precision:",round(np.average(precision_list),3),"recall:",round(np.average(recall_list),3),"F1:",round(np.average(f1_list),3))

In [ ]:
df_final = pd.DataFrame()
df_final["issue"] = test_data
df_final["y"] = ylabels
df_final["y_pred"] = yhatlabels
df_final
df_final.to_csv("../data/pretrained/Roberta_PS_pattern_result.csv",index =None)

In [ ]:
bert_model_name = 'electra_base'
tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

In [ ]:
precision_list = []
recall_list = []
f1_list = []
test_data = []
ylabels = []
yhatlabels = []
dropout_list = np.linspace(0.2, 0.6, 5)
epochs = 10

for i,(train_ds,val_ds,test_ds) in enumerate(zip(train_ds_etd_list, val_ds_etd_list, test_ds_etd_list)):  
    optimizer = create_optimizer(train_ds,epochs)
    max_clf = build_classifier_model(dropout = 0.2)
    max_clf.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)
    
    print(f'Training model with {tfhub_handle_encoder}')
    history = max_clf.fit(x=train_ds,
                        validation_data=val_ds,
                        verbose=0,
                        epochs=epochs)
    
    max_clf = None
    max_score = 0
    best_dropout = 0.2
    print(f'Training model with {tfhub_handle_encoder}')
    for rate in dropout_list:
        clf = build_classifier_model(dropout = rate)
        clf.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)
    
    
        history = clf.fit(x=train_ds,
                        validation_data=val_ds,
                        verbose=0,
                        epochs=epochs)
        
        curr_score = np.average(history.history["val_classifier1_f1_score"])
        if curr_score > max_score:
            max_score = curr_score
            max_clf = clf
            best_dropout = rate
    
    print("fold",(i+1),": best dropout rate is",best_dropout)
    test_etd_path = path+str(i)+"/test/test_bert_etd/"
    df_test = pd.read_csv(path+str(i)+"/test.csv")
    test_files = []
    for test_folder in class_list:
        file_path = test_etd_path+test_folder+"/"
        test_files.extend(sorted(os.listdir(file_path)))
    
    label_pred = evaluation_report(max_clf, test_ds,test_files,df_test)
    
    y_test = df_test["y_ETD"].values
    precison = mt.precision_score(y_test, label_pred)
    recall = mt.recall_score(y_test, label_pred)
    score = mt.f1_score(y_test, label_pred)
    
    test_data.extend(list(df_test["issue_clean"].values))
    ylabels.extend(list(y_test))
    yhatlabels.extend(list(label_pred))
    
    precision_list.append(precison)
    recall_list.append(recall)
    f1_list.append(score)

print("precision:",round(np.average(precision_list),3),"recall:",round(np.average(recall_list),3),"F1:",round(np.average(f1_list),3))

In [ ]:
df_final = pd.DataFrame()
df_final["issue"] = test_data
df_final["y"] = ylabels
df_final["y_pred"] = yhatlabels
df_final
df_final.to_csv("../data/pretrained/Electra_ETD_pattern_result.csv",index =None)

In [ ]:
precision_list = []
recall_list = []
f1_list = []
test_data = []
ylabels = []
yhatlabels = []
dropout_list = np.linspace(0.2, 0.6, 5)
epochs = 10

for i,(train_ds,val_ds,test_ds) in enumerate(zip(train_ds_ps_list, val_ds_ps_list, test_ds_ps_list)):
    optimizer = create_optimizer(train_ds,epochs)
    max_clf = build_classifier_model(dropout = 0.2)
    max_clf.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)
    
    print(f'Training model with {tfhub_handle_encoder}')
    history = max_clf.fit(x=train_ds,
                        validation_data=val_ds,
                        verbose=0,
                        epochs=epochs)
    max_clf = None
    max_score = 0
    best_dropout = 0.2
    print(f'Training model with {tfhub_handle_encoder}')
    for rate in dropout_list:
        clf = build_classifier_model(dropout = rate)
        clf.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)
    
    
        history = clf.fit(x=train_ds,
                        validation_data=val_ds,
                        verbose=0,
                        epochs=epochs)
        
        curr_score = np.average(history.history["val_classifier1_f1_score"])
        if curr_score > max_score:
            max_score = curr_score
            max_clf = clf
            best_dropout = rate
    
    print("fold",(i+1),": best dropout rate is",best_dropout)
    test_etd_path = path+str(i)+"/test/test_bert_ps/"
    df_test = pd.read_csv(path+str(i)+"/test.csv")
    test_files = []
    for test_folder in class_list:
        file_path = test_etd_path+test_folder+"/"
        test_files.extend(sorted(os.listdir(file_path)))
    
    label_pred = evaluation_report(max_clf, test_ds,test_files,df_test)
    
    y_test = df_test["y_PS"].values
    precison = mt.precision_score(y_test, label_pred)
    recall = mt.recall_score(y_test, label_pred)
    score = mt.f1_score(y_test, label_pred)
    
    test_data.extend(list(df_test["issue_clean"].values))
    ylabels.extend(list(y_test))
    yhatlabels.extend(list(label_pred))
    
    precision_list.append(precison)
    recall_list.append(recall)
    f1_list.append(score)

print("precision:",round(np.average(precision_list),3),"recall:",round(np.average(recall_list),3),"F1:",round(np.average(f1_list),3))

In [ ]:
df_final = pd.DataFrame()
df_final["issue"] = test_data
df_final["y"] = ylabels
df_final["y_pred"] = yhatlabels
df_final
df_final.to_csv("../Data/chat_pattern/result/validate_new3/pretrained/Electra_PS_pattern_result.csv",index =None)

In [ ]:
f1_list

### Plot the accuracy and loss over time

Based on the `History` object returned by `model.fit()`. You can plot the training and validation loss for comparison, as well as the training and validation accuracy:

In [ ]:
history_dict = history.history
print(history_dict.keys())

acc = history_dict['balanced_accuracy']
val_acc = history_dict['val_balanced_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()

plt.subplot(2, 1, 1)
# r is for "solid red line"
plt.plot(epochs, loss, 'r', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
# plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

In this plot, the red lines represent the training loss and accuracy, and the blue lines are the validation loss and accuracy.

## Export for inference

Now you just save your fine-tuned model for later use.

In [ ]:
dataset_name = 'test'
saved_model_path = 'sen_bert_m_models/{}_bert'.format(dataset_name.replace('/', '_'))

classifier_model.save(saved_model_path, include_optimizer=False)

Let's reload the model, so you can try it side by side with the model that is still in memory.

In [ ]:
reloaded_model = tf.saved_model.load(saved_model_path)

Here you can test your model on any sentence you want, just add to the examples variable below.

In [ ]:
def infer_by_cluster(text_list, tag_list):
    
    text_list = [html_Filter(sentence) for sentence in text_list]
    text_list = [remove_non_ascii(sentence) for sentence in text_list]
    text = tf.constant(text_list)
    
    feature = vectorizer.fit_transform(tag_list).toarray()
    feature = tf.constant(feature, dtype=tf.float32)
    
    input_dict = {
        "sentences": text,
        "tags": feature
    }

    y_prob = reloaded_model(input_dict, training=False)

    predicted_category_index = [np.argmax(y_prob[i]) for i in range(len(y_prob))]
    y_pred = [class_list[index] for index in predicted_category_index]
    
    return y_pred

In [ ]:
examples = [
        "How to convert nested setTimeouts to promises <p>Using rsvp.js or any other promises/A+ implementation how can I turn code like...</p> <pre><code>console.log('step 1'); setTimeout(function() { console.log('step 2'); setTimeout(function() { console.log('step 3'); }, 100); }, 300); </code></pre> <p>into a promises implementation?</p>",# this is the same sentence tried earlier
        "How to convert a char array to a uint16_t by casting type pointer? <pre><code>char bytes[2]; bytes[0] = 0; //0x00 bytes[1] = 24; //0x18 uint16_t* ptrU16 = (uint16_t*)bytes; // I expect it points to the memory block: 0x18 cout &lt;&lt; *ptrU16 &lt;&lt; endl;  // I expect 24, but it is 6144 </code></pre> <p>What's wrong in my code?</p>"
]
    
tag_list = [
        "<javascript><promise><rsvp.js><rsvp-promise>",
        "<c++><casting><uint16>"
]

In [ ]:
result = infer_by_cluster(examples,tag_list)

In [ ]:
result